# Features Seba

La idea de esta notebook es crear los siguientes features:

 - [screen_resolution](#screen_resolution)
 - [cantidad de eventos por categoría](#cantidad-de-eventos-por-categoría)
 - [cantidad de eventos](#cantidad-de-eventos)
 - [fechas](#fechas)
 - [precios](#precios)
     - [precios por modelo](#precios-por-modelo)
 - [diferencia de precios](#diferencia-de-precios) 
 - [Distancias a ciudades](#Distancias-a-ciudades)

In [1]:
from sklearn.feature_extraction import FeatureHasher
import requests, json, os, re

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

limpieza_path = '"{}"'.format(os.path.join(current_folder, '..', 'Limpieza.ipynb'))
modelos_a_buscar_path = os.path.join(current_folder, '..', 'modelos_a_buscar.csv')
precios_a_buscar_path = os.path.join(current_folder, '..', 'precios_a_buscar.csv')
precios_resultado_path = os.path.join(current_folder, '..', 'precios_resultado.csv')
coordenadas_path = os.path.join(current_folder, '..', 'fiuba-trocafone-tp2-final-set', 'BR.txt')

In [3]:
%run $limpieza_path
df = get_clean_df()

In [4]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


#### screen_resolution

Para este atributos se utilizó el promedio entre los distintos valores, sin tener en cuenta los nulos.

In [5]:
df_features['screen_resolution_height mean'] = df.groupby('person')['screen_resolution_height'].mean()
df_features['screen_resolution_width mean'] = df.groupby('person')['screen_resolution_width'].mean()
df_features['screen_resolution_height std'] = df.groupby('person')['screen_resolution_height'].std()
df_features['screen_resolution_width std'] = df.groupby('person')['screen_resolution_width'].std()

#### cantidad de eventos por categoría

La idea de este feature es tener una columna por categoría que indique la cantidad de eventos realizados por usuario en dicha categoría.

In [6]:
events_per_category_count = df.groupby('person')['event'].value_counts()
events_per_category_count.head()

person    event          
0008ed71  checkout             3
          visited site         2
          generic listing      1
00091926  viewed product     372
          visited site        34
Name: event, dtype: int64

In [7]:
events_per_category = events_per_category_count.unstack().fillna(0)
events_per_category.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0


In [8]:
df_features = df_features.merge(events_per_category, how='inner', on='person')

Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [9]:
df_features.shape[0] == df['person'].unique().shape[0]

True

#### cantidad de eventos

In [10]:
df_features['event_count'] = df.groupby('person')['event'].count()

In [11]:
df_features.head()

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count
person,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,9
ad93850f,640.0,360.0,0.0,0.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0,65
0297fc1e,640.0,360.0,0.0,0.0,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0,567
2d681dd8,768.0,1360.0,0.0,0.0,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0,26
cccea85e,864.0,1536.0,0.0,0.0,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0,836


#### fechas

Calculo el promedio y desviación estándar de días hasta el 31-05

In [12]:
df['dias hasta el 31-05'] = (df['timestamp'].max() - df['timestamp']).dt.days

In [13]:
days_mean_std = df.groupby('person')['dias hasta el 31-05'].agg(['mean', 'std'])
days_mean_std.rename(columns={'mean':'days until 31-05 mean', 'std':'days until 31-05 std'}, inplace=True)

In [14]:
df_features = df_features.merge(days_mean_std, how='inner', on='person')

In [15]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

También vamos a hacer un feature hashing de las fechas en la que un usuario tiene un evento en la página.

In [16]:
df['timestamp str'] = df['timestamp'].astype('str')
df['timestamp days str'] = df['timestamp str'].str.slice(0,10)
lista_timestamps = df.groupby('person').agg({'timestamp days str':list})

In [17]:
h = FeatureHasher(n_features=50, input_type='string')
f = h.transform(lista_timestamps['timestamp days str'])
features = pd.DataFrame(data=f.toarray()).add_prefix('feature_hashing_timestamp_days_')
features.index = lista_timestamps.index

In [18]:
df_features = df_features.merge(features, how='inner', on='person')
assert(df_features.shape[0] == df['person'].unique().shape[0])

#### precios

A través de Web scraping se obtuvieron los precios de los distintos modelos de trocafone al día 16/11/18 y se guardaron el archivo "precios.csv".

Para ello primero se guardó en el csv los modelos que había que buscar.

In [19]:
precios_a_buscar = df.groupby(['model']).first().index.to_frame(index=False)

In [20]:
precios_a_buscar.head()

,model
0,Asus Live
1,Asus Zenfone 2
2,Asus Zenfone 2 Deluxe
3,Asus Zenfone 2 Laser
4,"Asus Zenfone 2 Laser 6"""


In [21]:
pd.DataFrame(precios_a_buscar['model'].unique(), columns=['model']).set_index('model').to_csv(modelos_a_buscar_path)

Luego se realizó el scraping y el resultado fue el siguiente:

In [22]:
precios = pd.read_csv(precios_resultado_path, index_col='sku')
precios.head()

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
71.0,iPhone 4G,1.0,Preto,15.0,8GB,71.0,8.0,5.0,used,False,iPhone 4G 8GB Preto,Excelente,...,False,54.0,0.0,True,12.0,26.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#8-iphone-4g-8gb-preto-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
74.0,iPhone 4G,1.0,Preto,15.0,8GB,74.0,8.0,6.0,used,False,iPhone 4G 8GB Preto,Muito Bom,...,False,58.0,1.0,True,12.0,24.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#8-iphone-4g-8gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
80.0,iPhone 4G,2.0,Branco,15.0,8GB,80.0,9.0,6.0,used,False,iPhone 4G 8GB Branco,Muito Bom,...,False,58.0,0.0,True,12.0,24.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#9-iphone-4g-8gb-branco-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
81.0,iPhone 4G,2.0,Branco,15.0,8GB,81.0,9.0,5.0,used,False,iPhone 4G 8GB Branco,Excelente,...,False,54.0,0.0,True,12.0,26.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#9-iphone-4g-8gb-branco-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
87.0,iPhone 4G,1.0,Preto,16.0,16GB,87.0,10.0,6.0,used,False,iPhone 4G 16GB Preto,Muito Bom,...,False,54.0,1.0,True,12.0,28.25,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#10-iphone-4g-16gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 16GB Preto em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."


Chequeamos que todos los modelos tengan datos.

In [23]:
precios[precios['price_current'].isna()].shape[0] == 0

False

Vemos sobre cuales no encontramos resultados.

In [24]:
no_encontrados = precios[precios['price_current'].isna()]['model'].unique()
no_encontrados

array(['iPhone 5', 'iPhone 6S Plus', 'Samsung Galaxy Note 8',
       'Samsung Galaxy A7 2017'], dtype=object)

In [25]:
precios[precios['price_current'].isna()].index.unique()

Float64Index([5925.0, 5926.0, 7088.0, 14296.0, 14734.0], dtype='float64', name='sku')

Sobre estos 5 sku's no tenemos datos, pero sí sobre otros sku's que corresponden a los mismos modelos.

In [26]:
precios.loc[precios['model'].isin(no_encontrados)].head()

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
171.0,iPhone 5,1.0,Preto,16.0,16GB,171.0,22.0,6.0,used,False,iPhone 5 16GB Preto,Muito Bom,...,False,50.0,0.0,True,12.0,49.92,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
172.0,iPhone 5,1.0,Preto,16.0,16GB,172.0,22.0,5.0,used,False,iPhone 5 16GB Preto,Excelente,...,False,45.0,0.0,True,12.0,54.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
178.0,iPhone 5,2.0,Branco,16.0,16GB,178.0,23.0,6.0,used,False,iPhone 5 16GB Branco,Muito Bom,...,False,50.0,1.0,True,12.0,49.92,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
179.0,iPhone 5,2.0,Branco,16.0,16GB,179.0,23.0,5.0,used,False,iPhone 5 16GB Branco,Excelente,...,False,45.0,0.0,True,12.0,54.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
185.0,iPhone 5,1.0,Preto,17.0,32GB,185.0,24.0,6.0,used,False,iPhone 5 32GB Preto,Muito Bom,...,False,47.0,1.0,True,12.0,61.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#24-iphone-5-32gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 32GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."


Por ello vamos a buscar las características de estos sku's y completaremos con los que son parecidos (del mismo modelo).

Para los **iPhone 5**.

In [27]:
iphone5_skus = precios[precios['price_current'].isna() & (precios['model']=='iPhone 5')].index
iphone5_skus

Float64Index([5925.0, 5926.0], dtype='float64', name='sku')

In [28]:
df[df['sku']==5925].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
20820,iPhone 5,Bueno - Sin Touch ID,16.0,Blanco


In [29]:
df[df['sku']==5926].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
16431,iPhone 5,Bueno - Sin Touch ID,16.0,Negro


Por lo tanto vamos a elegir los modelos que más se parezcan a estos.

In [30]:
precios[(precios['model']=='iPhone 5') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Branco') & (precios['storage_name']=='16GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
2710.0,iPhone 5,2.0,Branco,16.0,16GB,2710.0,23.0,15.0,used,False,iPhone 5 16GB Branco,Bom,...,False,55.0,0.0,True,12.0,44.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [31]:
precios.loc[5925] = precios.loc[2710]

In [32]:
precios[(precios['model']=='iPhone 5') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='16GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
2711.0,iPhone 5,1.0,Preto,16.0,16GB,2711.0,22.0,15.0,used,False,iPhone 5 16GB Preto,Bom,...,False,55.0,1.0,True,12.0,44.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."


In [33]:
precios.loc[5926] = precios.loc[2711]

Para el **iPhone 6S Plus**.

In [34]:
iphone6_sku = precios[precios['price_current'].isna() & (precios['model']=='iPhone 6S Plus')].index
iphone6_sku

Float64Index([7088.0], dtype='float64', name='sku')

In [35]:
df[df['sku']==iphone6_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
37798,iPhone 6S Plus,Bueno - Sin Touch ID,64.0,Gris Espacial


In [36]:
precios[(precios['model']=='iPhone 6S Plus') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Cinza espacial') & (precios['storage_name']=='64GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
7085.0,iPhone 6S Plus,9.0,Cinza espacial,18.0,64GB,7085.0,497.0,15.0,used,False,iPhone 6S Plus 64GB Cinza Espacial,Bom,...,False,48.0,1.0,True,12.0,161.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#497-iphone-6s-plus-64gb-cinza-espacial-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 6S Plus 64GB Cinza Espacial em ...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [37]:
precios.loc[iphone6_sku[0]] = precios.loc[7085]

Para el **Samsung Galaxy Note 8**.

In [38]:
note8_sku = precios[precios['price_current'].isna() & (precios['model']=='Samsung Galaxy Note 8')].index
note8_sku

Float64Index([14296.0], dtype='float64', name='sku')

In [39]:
df[df['sku']==note8_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
73,Samsung Galaxy Note 8,Nuevo,64.0,Negro


In [40]:
precios[(precios['model']=='Samsung Galaxy Note 8')]['condition_name'].unique()

array(['Bom', 'Muito Bom', 'Excelente', nan], dtype=object)

En este caso nos llevamos la sorpresa de no encontrar ningún celular con estado *nuevo*, por lo cual se complica la tarea de completar los datos.

In [41]:
precios[(precios['model']=='Samsung Galaxy Note 8') & (precios['condition_name']=='Excelente')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='64GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
13180.0,Samsung Galaxy Note 8,1.0,Preto,18.0,64GB,13180.0,936.0,5.0,used,False,Samsung Galaxy Note 8 64GB Preto,Excelente,...,False,54.0,0.0,True,12.0,166.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#936-samsung-galaxy-note-8-64gb-preto-excelente-5,https://www.trocafone.com/celulares/samsung/ga...,Comprar Samsung Galaxy Note 8 64GB Preto em of...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [42]:
precios.loc[note8_sku[0]] = precios.loc[13180]

Pero vamos a modificar ciertos atributos.

In [43]:
precios.at[note8_sku[0], 'cellphone_condition_type'] = 'new'
precios.at[note8_sku[0], 'condition_name'] = 'Novo'
precios.at[note8_sku[0], 'price_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'price_invoice_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'price_one_installment_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'stock'] = 0

Para el **Samsung Galaxy A7 2017**.

In [44]:
a7_sku = precios[precios['price_current'].isna() & (precios['model']=='Samsung Galaxy A7 2017')].index
a7_sku

Float64Index([14734.0], dtype='float64', name='sku')

In [45]:
df[df['sku']==a7_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
1041,Samsung Galaxy A7 2017,Nuevo,32.0,Negro


In [46]:
precios[(precios['model']=='Samsung Galaxy A7 2017')]['condition_name'].unique()

array(['Bom', 'Muito Bom', 'Excelente', nan], dtype=object)

Nuevamente no encontramos ningún celular con estado *nuevo*, y hacemos lo mismo que antes.

In [47]:
precios[(precios['model']=='Samsung Galaxy A7 2017') & (precios['condition_name']=='Excelente')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='32GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
10884.0,Samsung Galaxy A7 2017,1.0,Preto,17.0,32GB,10884.0,772.0,5.0,used,False,Samsung Galaxy A7 2017 Preto,Excelente,...,False,55.0,1.0,True,12.0,80.75,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#772-samsung-galaxy-a7-2017-preto-excelente-5,https://www.trocafone.com/celulares/samsung/ga...,Comprar Samsung Galaxy A7 2017 Preto em oferta...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [48]:
precios.loc[a7_sku[0]] = precios.loc[10884]

Pero vamos a modificar ciertos atributos.

In [49]:
precios.at[a7_sku[0], 'cellphone_condition_type'] = 'new'
precios.at[a7_sku[0], 'condition_name'] = 'Novo'
precios.at[a7_sku[0], 'price_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'price_invoice_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'price_one_installment_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'stock'] = 0

Volvemos a chequear que todos los modelos tengan datos.

In [50]:
assert(precios[precios['price_current'].isna()].shape[0] == 0)

Convierto los sku's de los precios en categorías para poder hacer el merge con el dataframe

In [51]:
precios.index = precios.index.astype('category')

Hacemos el merge, y llenamos los nan para que sean incluidos en el groupby. (Esta solución deberíamos mejorarla de alguna forma). 

### precios por modelo

In [52]:
columna_f_h = 'model_x'

In [53]:
df_merged = df.merge(precios, how='left', on='sku')
df_merged[columna_f_h].cat.add_categories(['nan'], inplace=True)
df_merged[columna_f_h].fillna('nan', inplace=True)

In [54]:
grouped = df_merged.groupby(['person', columna_f_h]).agg({'price_current': 'mean'}).unstack()
grouped.columns = grouped.columns.droplevel()
assert(grouped.shape[0] == df_features.shape[0])
grouped.head()

model_x,Asus Live,Asus Zenfone 2,Asus Zenfone 2 Deluxe,Asus Zenfone 2 Laser,"Asus Zenfone 2 Laser 6""",Asus Zenfone 3 Max 32 GB,Asus Zenfone 3 Max 16 GB,Asus Zenfone 5,Asus Zenfone 6,Asus Zenfone Go,Asus Zenfone Selfie,LG X Screen,...,iPhone 5s,iPhone 6,iPhone 6 Plus,iPhone 6S,iPhone 6S Plus,iPhone 7,iPhone 7 Plus,iPhone 8,iPhone 8 Plus,iPhone SE,iPhone X,nan
person,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1199.000000,NaN,NaN
00091926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1447.0,1639.243902,1733.479167,2000.764706,2442.555556,3121.222222,NaN,3634.0,1065.666667,5919.0,NaN
00091a7a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1839.0,NaN,1679.000000,NaN,NaN,NaN,NaN,NaN,1199.000000,NaN,NaN
000ba417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1139.0,NaN,NaN,NaN,NaN,2879.000000,NaN,NaN,NaN,NaN,NaN,NaN
000c79fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2469.000000,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que hay ciertos usuarios para los cuales no tenemos información sobre el precio de los modelos que vieron. Para estos vamos a llenar todos sus valores con nulos (o ceros,  dado que en este caso luego los nulos los transformamos en ceros). Luego devolvemos los valores en forma de diccionario para el feature hashing.

In [55]:
d = grouped.fillna(0).to_dict('records')

Voy a trabajar con tantos 'features' como modelos haya. Voy a obtener un aproximado de la siguiente manera.

In [56]:
h = FeatureHasher(n_features=10)
f = h.transform(d)
features = pd.DataFrame(data=f.toarray()).add_prefix('precios_por_modelo_')
features.index = grouped.index

In [57]:
assert(features.shape[0] == df_features.shape[0])
features.head()

,precios_por_modelo_0,precios_por_modelo_1,precios_por_modelo_2,precios_por_modelo_3,precios_por_modelo_4,precios_por_modelo_5,precios_por_modelo_6,precios_por_modelo_7,precios_por_modelo_8,precios_por_modelo_9
person,,,,,,,,,,
0008ed71,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-759.000000,-659.000000,-1199.000000
00091926,-3974.984127,-519.000000,-1135.0,-529.0,379.000000,-3447.764706,497.666667,-9797.479167,-2720.888889,494.291521
00091a7a,0.000000,0.000000,0.0,0.0,0.000000,-1839.000000,0.000000,-1679.000000,0.000000,-1199.000000
000ba417,-3186.333333,-836.142857,-449.0,409.0,1035.636364,0.000000,-1019.142857,-325.000000,-825.500000,-434.360119
000c79fe,-2469.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
df_features = df_features.merge(features, how='inner', on='person')
assert(df_features.shape[0] == df['person'].unique().shape[0])

## diferencia de precios

In [59]:
df_merged['price_difference'] = df_merged['price_current']/df_merged['price_new']

## Distancias a ciudades

En este feature vamos a calcular el promedio de las distancias de las visitas de los usuarios a las $k$ ciudades con más conversiones. Idealmente $k=total\_de\_ciudades$ pero esto haría que la cantidad de columnas resultara inmanejable, por lo que decidimos utilizar las que suponemos que aportan mayor cantidad de información.

In [60]:
k = 5

#### Obtener coordenadas

Para calcular las distancias primero obtenemos las coordenadas de cada ciudad. Para ello se descargó un set de datos de http://www.geonames.org con las ciudades de Brasil (http://download.geonames.org/export/dump/BR.zip) que contiene el nombre de las ciudades y sus coordenadas. No vamos a obtener las coordenadas de las ciudades fuera de Brasil dado que , como se vió en el tp1, en dichas ciudades las conversiones son muy pocas.
El dataset de coordenadas contiene los siguientes atributos:
- geonameid         : integer id of record in geonames database
- name              : name of geographical point (utf8) varchar(200)
- asciiname         : name of geographical point in plain ascii characters, varchar(200)
- alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
- latitude          : latitude in decimal degrees (wgs84)
- longitude         : longitude in decimal degrees (wgs84)
- feature class     : see http://www.geonames.org/export/codes.html, char(1)
- feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
- country code      : ISO-3166 2-letter country code, 2 characters
- cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
- admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
- admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
- admin3 code       : code for third level administrative division, varchar(20)
- admin4 code       : code for fourth level administrative division, varchar(20)
- population        : bigint (8 byte int) 
- elevation         : in meters, integer
- dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
- timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
- modification date : date of last modification in yyyy-MM-dd format

De todas estas nosotros utilizaremos únicamente name, asciiname, population, latitude y longitude.

In [61]:
columnas = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code',
            'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation',
            'dem', 'timezone', 'modification date']
coordenadas = pd.read_csv(coordenadas_path, sep='\t', header=None, names=columnas, usecols=['name', 'asciiname', 'feature class','latitude', 'longitude', 'population'])

Este set de datos viene con varias entradas por ciudad con distintas ubicaciones dentro de las mismas:

In [62]:
coordenadas[coordenadas['name'] == 'Rio de Janeiro'].head()

,name,asciiname,latitude,longitude,feature class,population
33210,Rio de Janeiro,Rio de Janeiro,-22.25000,-42.50000,A,15993583
33211,Rio de Janeiro,Rio de Janeiro,-22.90642,-43.18223,P,6023699
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H,0
42180,Rio de Janeiro,Rio de Janeiro,-11.84252,-45.17394,H,0
75734,Rio de Janeiro,Rio de Janeiro,-22.92008,-43.33069,A,6323037


Esto se debe a que ciertos registros se refieren a ciudades, y otros parte del terreno que llevan el mismo nombre. Un ejemplo claro es el siguiente registro:

In [63]:
coordenadas.iloc[[42179]]

,name,asciiname,latitude,longitude,feature class,population
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H,0


Si buscamos esas coordendas encontraremos que se refiere al rio 'Rio de Janeiro'. Es por esto que a su vez utilizaremos el atributo 'feature class' y tomaremos solo aquellas con el valor 'P' (que indica registros de ciudades, pueblos, etc.).

In [64]:
coordenadas = coordenadas[coordenadas['feature class'] == 'P']

Sin embargo notamos que sigue habiendo varias entradas repetidas:

In [65]:
coordenadas.shape[0] == coordenadas['name'].unique().size

False

Para quedarnos con una sola tomaremos como criterio la que tenga una población mayor.

In [66]:
indices = coordenadas.groupby('name', as_index=True)['population'].idxmax()
coordenadas = coordenadas.loc[indices]

Chequeamos que ahora los nombres sean únicos:

In [67]:
assert(coordenadas.shape[0] == coordenadas['name'].unique().size)

Y dado que ya no necesitamos los atributos 'population' ni 'feature class', los borramos:

In [68]:
coordenadas = coordenadas.drop(columns=['population', 'feature class'])

In [69]:
coordenadas.head(3)

,name,asciiname,latitude,longitude
23587,Aba,Aba,-6.71667,-37.98333
23577,Aba da Serra,Aba da Serra,-5.91667,-39.51667
104680,Abacabal,Abacabal,-6.64503,-69.83667


#### Merge de coordenadas con el dataframe

Primero nos vamos a quedar con las ciudades de Brasil

In [70]:
cities = pd.DataFrame(data=list(df[(df['country']=='Brazil') & (~df['city'].isna())]['city'].unique()))
cities.columns = ['city']

Ahora buscamos las coordenadas por 'name' y 'asciiname'.

In [71]:
cities_with_coord_by_name = cities.merge(coordenadas, how='inner', left_on='city', right_on='name').drop(columns=['name', 'asciiname'])
cities_with_coord_by_asciiname = cities.merge(coordenadas, how='inner', left_on='city', right_on='asciiname').drop(columns=['name', 'asciiname'])

In [72]:
cities_with_coord_by_name.head(2)

,city,latitude,longitude
0,São Bernardo do Campo,-23.69389,-46.56500
1,Florianópolis,-27.59667,-48.54917


In [73]:
cities_with_coord_by_asciiname.head(2)

,city,latitude,longitude
0,Nova Esperanca,-5.37139,-36.66361
1,Alto Parana,-23.12889,-52.31889


In [74]:
cities_with_coord = cities_with_coord_by_name.append(cities_with_coord_by_asciiname).groupby('city').apply(lambda x: x.iloc[1] if np.isnan(x.iloc[0]).any() else x.iloc[0])

In [75]:
cities_with_coord_merged = cities.merge(cities_with_coord, how='left', on='city')

Vemos que queda una ciudad sin coordenadas

In [76]:
cities_with_coord_merged[cities_with_coord_merged.isna().any(axis='columns')]

,city,latitude,longitude
333,São Miguel do Oeste,NaN,NaN


Y lo podemos llenar a mano.

In [77]:
cities_with_coord_merged.loc[cities_with_coord_merged['city'] == 'São Miguel do Oeste', ['latitude', 'longitude']] = -26.71868, -53.5194

In [78]:
assert(cities_with_coord_merged[cities_with_coord_merged.isna().any(axis='columns')].shape[0] == 0)

Ahora podemos hacer un merge a partir de las ciudades con el dataframe original.

In [79]:
df_with_coord = df.merge(cities_with_coord_merged, how='inner', on='city')
df_with_coord.head(2)

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,...,country,device_type,screen_resolution,operating_system_version,browser_version,screen_resolution_width,screen_resolution_height,dias hasta el 31-05,timestamp str,timestamp days str,latitude,longitude
0,2018-05-10 22:34:50,visited site,4640420b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Brazil,Smartphone,320x570,Android 5.0.2,Chrome Mobile 66.0,320.0,570.0,21,2018-05-10 22:34:50,2018-05-10,-5.37139,-36.66361
1,2018-05-15 02:39:45,visited site,4640420b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Brazil,Smartphone,320x570,Android 5.0.2,Chrome Mobile 66.0,320.0,570.0,16,2018-05-15 02:39:45,2018-05-15,-5.37139,-36.66361


#### k ciudades con más ventas

Voy a ver de qué ciudad es cada usuario. Si bien varios usuarios tienen registros desde más de una ciudad, voy a suponer que son de la ciudad de la que tienen más registros.

In [80]:
# parece que no hay un groupby.mode, y esto es lo más óptimo que encontré
city_by_user = df.groupby('person')['city'].value_counts().groupby('person').head(1)
city_by_user.rename('cantidad de visitas', inplace=True)
city_by_user = city_by_user.reset_index(level=1)[['city']]
city_by_user.head(3)

,city
person,
00091926,Carlos Barbosa
00091a7a,Salvador
000ba417,Rio Grande da Serra


Vemos que hay usuarios que no tienen ni un registro con una ciudad. En este paso no nos afectan al cálculo dado que los vamos a ignorar tanto si hicieron una conversión como sino. Más adelante tendremos que decidir qué hacer con ellos. 

In [81]:
city_by_user.shape[0], df['person'].unique().shape[0]

(33156, 38829)

In [82]:
eventos_conversion = df[df['event'] == 'conversion'][['person']]
conversions_by_city = eventos_conversion.merge(city_by_user, how='inner', on='person')
conversions_by_city.head(3)

,person,city
0,49c19e32,São Paulo
1,49c19e32,São Paulo
2,49c19e32,São Paulo


In [83]:
cities = pd.DataFrame(index=conversions_by_city['city'].value_counts().iloc[:k].index)
cities.index.name = 'city'
cities

""
city
São Paulo
Salvador
Rio de Janeiro
Belo Horizonte
Brasília


In [84]:
cities = cities.merge(cities_with_coord_merged, how='inner', on='city')
cities

,city,latitude,longitude
0,São Paulo,-23.54750,-46.63611
1,Salvador,-12.97111,-38.51083
2,Rio de Janeiro,-22.90642,-43.18223
3,Belo Horizonte,-19.92083,-43.93778
4,Brasília,-15.77972,-47.92972


#### distancia hasta las k ciudades con más ventas

Distancia euclidiana. 

In [85]:
for city in cities.itertuples():
    df_with_coord['distancia a '+city[1]] = ( (df_with_coord['latitude']-city[2]).pow(2) + (df_with_coord['longitude']-city[3]).pow(2) ).pow(.5)

In [86]:
df_with_coord.head(3)

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,...,screen_resolution_width,screen_resolution_height,dias hasta el 31-05,timestamp str,timestamp days str,latitude,longitude,distancia a São Paulo,distancia a Salvador,distancia a Rio de Janeiro,distancia a Belo Horizonte,distancia a Brasília
0,2018-05-10 22:34:50,visited site,4640420b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,320.0,570.0,21,2018-05-10 22:34:50,2018-05-10,-5.37139,-36.66361,20.732142,7.820995,18.707477,16.266522,15.338141
1,2018-05-15 02:39:45,visited site,4640420b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,320.0,570.0,16,2018-05-15 02:39:45,2018-05-15,-5.37139,-36.66361,20.732142,7.820995,18.707477,16.266522,15.338141
2,2018-05-18 19:03:37,visited site,4640420b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,800.0,1280.0,13,2018-05-18 19:03:37,2018-05-18,-5.37139,-36.66361,20.732142,7.820995,18.707477,16.266522,15.338141


Hago el merge final y lleno los nulls.

In [87]:
for city in cities.itertuples():
    dist = df_with_coord.groupby('person')[['distancia a '+city[1]]].mean()
    dist.fillna(dist.max()**3, inplace=True)
    df_features = df_features.merge(dist, how='inner', on='person')

## Modelo más problable que compre

Vamos a generar un modelo muy sencillo que nos diga qué modelo es más probable que compre el usuario. Este es un problema que se podría encarar con un algoritmo de machine learning, pero la idea ahora es tomar un modelo sencillo que puede que nos sirva como feature. Igualmente en la mayoría de los casos no podríamos medir que tan acertados estamos dado que a menos que un usuario compre un producto, no resulta posible saber cual era el más probable que compre. Esto resulta análogo a penalizar a un algoritmo por no recomendar la pelicula que un usuario vió, cuando la recomendación del algoritmo puede haber sido excelente. 

Como primera aproximación vamos a tomar la moda para cada usuario. Notar que en caso de empates nos quedamos con el primero (¿qué significará el primero?).

In [88]:
modelo_mas_probable = df.groupby('person')['model'].value_counts().groupby('person').head(1)
modelo_mas_probable.name = 'cantidad de eventos para el modelo más probable'
modelo_mas_probable = modelo_mas_probable.reset_index(level=1)
modelo_mas_probable.head()

,model,cantidad de eventos para el modelo más probable
person,,
0008ed71,LG G4 H818P,1
00091926,iPhone 6S,96
00091a7a,iPhone 6,1
000ba417,Samsung Galaxy A3 Duos,32
000c79fe,iPhone 7,4


In [89]:
df_features = df_features.merge(modelo_mas_probable[['cantidad de eventos para el modelo más probable']], how='left', on='person')
df_features.loc[:, 'cantidad de eventos para el modelo más probable'].fillna(0, inplace=True)
df_features.head(2)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,precios_por_modelo_4,precios_por_modelo_5,precios_por_modelo_6,precios_por_modelo_7,precios_por_modelo_8,precios_por_modelo_9,distancia a São Paulo,distancia a Salvador,distancia a Rio de Janeiro,distancia a Belo Horizonte,distancia a Brasília,cantidad de eventos para el modelo más probable
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.546408,9.800935,1.232506,2.590130,8.330024,5.0
ad93850f,640.0,360.0,0.0,0.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,...,529.0,0.0,0.0,0.0,0.0,-1269.0,0.000000,13.337174,3.512872,4.520367,7.874759,18.0


In [90]:
assert(df_features[df_features['cantidad de eventos para el modelo más probable'].isna()].shape[0] == 0)

Por alguna razón existen usuarios que entran a Trocafone pero no miran ninguna página. Puede que sólo visiten la página principal. A estos usuarios hay que tratarlos de forma especial más adelante.

In [91]:
persons = df['person'].unique()
persons[~persons.isin(modelo_mas_probable.index)]

[df80a783, 7a3184e8, f33e2cc5, fb3c6e61, 5437de8d, ..., 6f7632db, a1c2a901, ed3f80d7, 2adb3684, 39fa45b6]
Length: 573
Categories (38829, object): [4886f805, ad93850f, 0297fc1e, 2d681dd8, ..., 92f2d94b, d8d6b0da, 40bf23ab, 80aea0a0]

### sku más probable

Vamos a encontrar el **sku** más visto dentro del modelo más visto.

In [92]:
eventos_del_modelo_mas_probable = df.merge(modelo_mas_probable, how='inner', on=['person', 'model'])
eventos_del_modelo_mas_probable['person'].cat.remove_unused_categories(inplace=True)
sku_mas_probable = eventos_del_modelo_mas_probable.groupby('person')['sku'].value_counts().groupby('person').head(1)
sku_mas_probable.name = 'cantidad de eventos para el sku más probable'
sku_mas_probable = sku_mas_probable.reset_index(level=1)
sku_mas_probable.columns = ['sku mas probable', 'cantidad de eventos para el sku más probable']
sku_mas_probable.head()

,sku mas probable,cantidad de eventos para el sku más probable
person,,
0008ed71,7505.0,1
00091926,6902.0,13
00091a7a,1357.0,1
000ba417,2988.0,12
000c79fe,9944.0,4


In [93]:
df_features = df_features.merge(sku_mas_probable, how='left', on='person')
df_features.loc[:, 'cantidad de eventos para el sku más probable'].fillna(0, inplace=True)
df_features.loc[:, 'sku mas probable'].fillna(sku_mas_probable['sku mas probable'].max()+1, inplace=True)
df_features.head(2)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,precios_por_modelo_6,precios_por_modelo_7,precios_por_modelo_8,precios_por_modelo_9,distancia a São Paulo,distancia a Salvador,distancia a Rio de Janeiro,distancia a Belo Horizonte,distancia a Brasília,cantidad de eventos para el modelo más probable,sku mas probable,cantidad de eventos para el sku más probable
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,4.546408,9.800935,1.232506,2.590130,8.330024,5.0,9288.0,4.0
ad93850f,640.0,360.0,0.0,0.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,...,0.0,0.0,0.0,-1269.0,0.000000,13.337174,3.512872,4.520367,7.874759,18.0,304.0,4.0


In [94]:
assert(df_features[df_features['cantidad de eventos para el sku más probable'].isna()].shape[0] == 0)
assert(df_features[df_features['sku mas probable'].isna()].shape[0] == 0)

## Features relacionados al sku más probable

En está sección vamos a aprovechar la información en el dataframe precios, y agregar otras columnas relacionadas al sku más probable.

In [95]:
features_precios = precios[['color_id', 'cellphone_condition_id', 'enable_membership', 'featured', 'discount', 'stock', 'super_discount', 'installments', 'installment_value', 'interest', 'price_new', 'price_current']].copy(deep=True)
features_precios['super_discount'] = features_precios['super_discount'].astype('int32', copy=True)
features_precios['enable_membership'] = features_precios['enable_membership'].astype('int32', copy=True)
features_precios['featured'] = features_precios['featured'].astype('int32', copy=True)
features_precios.index = features_precios.index.astype('category', copy=True)
df_features['sku mas probable'] = df_features['sku mas probable'].astype('category', copy=True)

In [96]:
df_features = df_features.reset_index().merge(features_precios, how='left', left_on='sku mas probable', right_on='sku').set_index('person')
df_features.head(3)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,color_id,cellphone_condition_id,enable_membership,featured,discount,stock,super_discount,installments,installment_value,interest,price_new,price_current
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,8.0,5.0,0,0,50.0,1.0,1,12.0,62.42,0.0,1499.0,749.0
ad93850f,640.0,360.0,0.0,0.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,...,9.0,6.0,0,0,45.0,0.0,1,12.0,73.25,0.0,1619.0,879.0
0297fc1e,640.0,360.0,0.0,0.0,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,...,8.0,15.0,0,0,59.0,0.0,1,12.0,91.58,0.0,2699.0,1099.0


Ahora tenemos que ver que hacer con los nulos.

In [97]:
df_features.loc[:, 'color_id'].fillna(df_features['color_id'].max()+1, inplace=True)
df_features.loc[:, 'cellphone_condition_id'].fillna(df_features['cellphone_condition_id'].max()+1, inplace=True)
df_features.loc[:, 'enable_membership'].fillna(0, inplace=True)
df_features.loc[:, 'featured'].fillna(0, inplace=True)
df_features.loc[:, 'discount'].fillna(0, inplace=True)
df_features.loc[:, 'stock'].fillna(0, inplace=True)
df_features.loc[:, 'super_discount'].fillna(0, inplace=True)
df_features.loc[:, 'installments'].fillna(12, inplace=True)
df_features.loc[:, 'installment_value'].fillna(df_features['installment_value'].max()**4, inplace=True)
df_features.loc[:, 'interest'].fillna(0, inplace=True)
df_features.loc[:, 'price_new'].fillna(df_features['price_new'].mean(), inplace=True)
df_features.loc[:, 'price_current'].fillna(df_features['price_current'].mean(), inplace=True)
df_features.head()

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,color_id,cellphone_condition_id,enable_membership,featured,discount,stock,super_discount,installments,installment_value,interest,price_new,price_current
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,8.0,5.0,0,0,50.0,1.0,1,12.0,62.42,0.0,1499.0,749.0
ad93850f,640.0,360.0,0.0,0.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,...,9.0,6.0,0,0,45.0,0.0,1,12.0,73.25,0.0,1619.0,879.0
0297fc1e,640.0,360.0,0.0,0.0,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,...,8.0,15.0,0,0,59.0,0.0,1,12.0,91.58,0.0,2699.0,1099.0
2d681dd8,768.0,1360.0,0.0,0.0,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,...,1.0,15.0,0,0,53.0,0.0,1,12.0,42.42,0.0,1099.0,509.0
cccea85e,864.0,1536.0,0.0,0.0,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,...,1.0,6.0,0,0,49.0,0.0,1,12.0,46.58,0.0,1099.0,559.0


In [98]:
assert(df_features['color_id'].isna().sum() == 0)
assert(df_features['cellphone_condition_id'].isna().sum() == 0)
assert(df_features['enable_membership'].isna().sum() == 0)
assert(df_features['featured'].isna().sum() == 0)
assert(df_features['discount'].isna().sum() == 0)
assert(df_features['stock'].isna().sum() == 0)
assert(df_features['super_discount'].isna().sum() == 0)
assert(df_features['installments'].isna().sum() == 0)
assert(df_features['installment_value'].isna().sum() == 0)
assert(df_features['interest'].isna().sum() == 0)
assert(df_features['price_new'].isna().sum() == 0)
assert(df_features['price_current'].isna().sum() == 0)

#### diferencia de precios

In [99]:
df_features['diferencia de precio'] = df_features['price_new'] - df_features['price_current']

In [100]:
df_features['diferencia de precio porcentual'] = df_features['diferencia de precio']/df_features['price_current']